In [5]:
#импорт
import pandas as pd
import requests
import json
import time
import math
import vk
import collections
import os

import vkauth
#vk_auth.py
#auth = {"id_app":'' , "login":'', "password":''}

session = vk.AuthSession(vkauth.auth['id_app'], vkauth.auth['login'], vkauth.auth['password'])
vk_api = vk.API(session)

In [2]:
from functools import wraps 
from ipywidgets import IntProgress
from IPython.display import display

def logging_decorator(func):     
    
    progress = IntProgress(min=0, max=774559, value=0)
    display(progress)
    
    @wraps
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        if (wrapper.count % 10 == 0):
            progress.value = wrapper.count
        func(*args, **kwargs)
        
    wrapper.count = 0
    return wrapper

In [3]:
#задать сетку
def get_circle_centers(lower, upper, radius):
    """
    cover the search area with circles for radar search
    http://stackoverflow.com/questions/7477003/calculating-new-longtitude-latitude-from-old-n-meters
    :param lower: lower bound of area (westmost + southmost)
    :param upper: upper bound of area (eastmost + northmost)
    :param radius: specified radius, adapt for high density areas
    :return: list of circle centers that cover the area between lower/upper
    """
    r, coords = 6378, list()
    while lower[1] < upper[1]:
        tmp = lower[0]

        while tmp < upper[0]:
            coords.append([tmp, lower[1]])
            tmp += (0.25 / r) * (radius / math.pi)
        lower[1] += (0.25 / r) * (radius / math.pi) / math.cos(lower[00] * math.pi / radius)

    return coords

bounds = {'lower':{'lat': 59.808311, 'lng': 30.135219}, 'upper':{'lat': 60.087617, 'lng': 30.526297}}
circles = get_circle_centers([bounds["lower"]["lat"], bounds["lower"]["lng"]],  # southwest
                                       [bounds["upper"]["lat"], bounds["upper"]["lng"]],  # northeast
                                       150)


In [3]:
#запрос для каждого элемента сетки

deq = collections.deque(maxlen=4)
def trottling_request():
    deq.appendleft(time.time())
    if len(deq) == 4:
        # 3 запроса в секунду, если нужно - подождем
        time.sleep(max(1+deq[3]-deq[0], 0.2))
        

def getPlaces(lat, lon, radius=1, count=1000, offset=0):
    if count > 1000:
        raise Exception('Bad params: max of count = 1000')
        
    response = vk_api.places.search(latitude = lat, longitude = lon, radius = radius, count = count, offset = offset)
    
    return response
    
def allPlacesOffset(func, func_lat, func_lon):
    places = []
    place_count = -1
    offset = 0
    while place_count != len(places):
        trottling_request()
        
        try:
            response = func(func_lat, func_lon, radius = 1, count = 1000, offset = offset)
        except Exception:
            print("error")
            time.sleep(1)
            continue
          
        if place_count != response[0]:
            place_count = response[0]
        
        items = response[1:]
        offset = offset + len(items)
        
        places = places + items
    
    return places
        
def getCheckins(place_id, count=100, offset = 0):
    #host = 'http://api.vk.com/method'
    if count > 100:
        raise Exception('Bad params: max of count = 100')
    #response = requests.get('{host}/places.getCheckins?latitude={lat}&longitude={lon}&count={count}&offset={offset}&need_places=1'
    #                        .format(host=host,lat=lat, lon=lon, count=count, offset=offset))
    response = vk_api.places.getCheckins(place = place_id, count = count, offset = offset)
    
    return response        

def allChecinsCountOffset(func, place_id):
    checkins = []
    count_checins = -1
    offset = 0
    while count_checins != len(checkins): # posible endless loop for real vk api
        trottling_request()
        try:
            response = func(place_id,count=100, offset=offset)
        except Exception:
            print('error')
            time.sleep(1)
            continue
        
        if count_checins != response[0]:
            count_checins = response[0]
        
        items = response[1:]
        
        if len(items) == 0:
            return checkins
        
        offset = offset + len(items)
        
       
        #if dict_checkins | set(new_members_id) == set_members_id != set(): # without new members
         #   print('WARNING: break loop', count_members, len(set_members_id))
          #  break
        
        checkins = checkins + items

    return checkins

#all_checkins = []
#for lat, lng in circles:
#    checkins = allChecinsCountOffset(getCheckins, lat, lng)
#    all_checkins = all_checkins + checkins
    

In [5]:
all_places = []
for lat, lng in circles:
    places = allPlacesOffset(getPlaces, lat, lng)
    all_places = all_places + places

error


In [4]:
def checins_from_places(place_id):
    checkins = allChecinsCountOffset(getCheckins, place_id)
    for checkin in checkins:
        checkin['place_id'] = int(place_id)
    return checkins

In [5]:
places_df = pd.read_csv('places_drops.csv', encoding='windows-1251')

In [6]:
places_id_list = places_df.place_id.tolist()

In [7]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display
    
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)
    
    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [ ]:
all_checkins = []
iterator = 0
for place_id in log_progress(places_id_list[89649:], 10):
    if iterator % 1000 == 0:
        #дописать chekins в файл и очистить all_checkins
        with open('all_checkins_2', 'a+') as fout:
            for checkin in all_checkins:
                json.dump(checkin, fout)
                fout.write(os.linesep)
        all_checkins = []
        
    checkins = checins_from_places(place_id)
    all_checkins.extend(checkins)
    
    iterator = iterator + 1

In [75]:
for index, record in enumerate(places_id_list[1:20], 1):
    print(index, record)

1 7210724
2 14087516
3 18055509
4 14087516
5 18055509
6 12467862
7 15227239
8 8505920
9 15227239
10 10423297
11 10289996
12 7229933
13 17302260
14 8505920
15 13267460
16 5046513
17 7612518
18 11780429
19 17598710


In [28]:
checkins = checins_from_places(14087516)

In [46]:
with open('all_checkins', 'a') as fout:
    for checkin in all_checkins:
        print(type(checkin['place_id']))
        json.dump(checkin, fout)
        fout.write(os.linesep)

<class 'numpy.int64'>


TypeError: 14087516 is not JSON serializable

In [37]:
all_checkins

[{'date': 1517161955.0,
  'id': '349132_21264',
  'latitude': 59.81388,
  'longitude': 30.132488,
  'place_id': 14087516.0,
  'post_id': 21264.0,
  'text': 'Ну как же без фоток?)  #горнолыжныйкурорт_игора #сноуборд2018 #январь2018',
  'uid': 349132.0},
 {'date': 1516803938.0,
  'id': '37623783_6315',
  'latitude': 59.81388,
  'longitude': 30.132488,
  'place_id': 14087516.0,
  'post_id': 6315.0,
  'text': '🏂💫',
  'uid': 37623783.0},
 {'date': 1516561061.0,
  'id': '234591556_8010',
  'latitude': 59.81388,
  'longitude': 30.132488,
  'place_id': 14087516.0,
  'post_id': 8010.0,
  'text': 'Без фильтра)) Любимая Игора))',
  'uid': 234591556.0},
 {'date': 1515812470.0,
  'id': '263487492_211',
  'latitude': 59.81388,
  'longitude': 30.132488,
  'place_id': 14087516.0,
  'post_id': 211.0,
  'text': 'Ни горы, ни овраги и не лес.,,, ❄️🏂🚀',
  'uid': 263487492.0},
 {'date': 1515784976.0,
  'id': '244453510_2542',
  'latitude': 59.81388,
  'longitude': 30.132488,
  'place_id': 14087516.0,
  'pos

In [5]:
with open('all_checkins', 'r') as f:
    my_list = [json.loads(line) for line in f]

In [6]:
checkin_df = pd.DataFrame(my_list)

In [9]:
checkin_df.place_id.head(-1)

0         14087516
1         14087516
2         14087516
3         14087516
4         14087516
5         14087516
6         14087516
7         14087516
8         14087516
9         14087516
10        14087516
11        14087516
12        14087516
13        14087516
14        14087516
15        14087516
16        14087516
17        14087516
18        14087516
19        14087516
20        14087516
21        14087516
22        14087516
23        14087516
24        14087516
25        14087516
26        14087516
27        14087516
28        14087516
29        14087516
            ...   
834562    17325344
834563    17325344
834564    13849206
834565    13849206
834566    13849206
834567    13849206
834568    13849206
834569    13849206
834570    13849206
834571    16140781
834572    16140781
834573    16140781
834574    16140781
834575    16140781
834576    16140781
834577    16140781
834578    16140781
834579    16140781
834580    16140781
834581    16140781
834582    16140781
834583    16

In [8]:
places = checkin_df.place_id.drop_duplicates()

In [3]:
places.head(-1)

NameError: name 'places' is not defined

In [ ]:
all_places = pd.read_csv('places_petro.csv', encoding='windows_1251')

NameError: name 'all_places' is not defined

774559